In [1]:
from xcube.core.dsio import open_cube
from xcube.core.resample import resample_in_time
from xcube.core.geom import mask_dataset_by_geometry
from xcube.core.extract import get_cube_values_for_points

import IPython.display

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

xr.set_options(display_style="html")

import warnings
warnings.filterwarnings("ignore")

---

#### Index

This notebook contains four examples:

1. Inspect the CHL in the cube
2. Create monthly CHL aggregations for specific regions
3. Compute monthly CHL "anomalies" 
4. Compare Cube CHL with In-Situ CHL


#### Installation

For creating an `xcube` Python environment and installing `xcube` follow the instructions given in the [xcube's README](https://github.com/dcs4cop/xcube/blob/master/README.md).

Before using Jupyter Lab for the first time install the `jupyterlab` package and make sure the 
[Jupyter GeoJSON extension](https://www.npmjs.com/package/@jupyterlab/geojson-extension) is installed too:

```bash
(xcube) conda install -c conda-forge jupyterlab
(xcube) jupyter labextension install @jupyterlab/geojson-extension
```


---

## Inspect the CHL in the cube

Open demo data cube with CHL, TSM, and Turbidity for 2017:

In [2]:
cube = open_cube('https://s3.eu-central-1.amazonaws.com/dcs4cop/bc-olci-sns-l2c-2017_1x1024x1024.zarr', s3_kwargs=dict(anon=True))
cube

<xarray.Dataset>
Dimensions:         (bnds: 2, lat: 5632, lon: 10240, time: 1479)
Coordinates:
  * lat             (lat) float64 62.67 62.66 62.66 62.66 ... 48.01 48.0 48.0
    lat_bnds        (lat, bnds) float64 dask.array<chunksize=(5632, 2), meta=np.ndarray>
  * lon             (lon) float64 -16.0 -16.0 -15.99 ... 10.66 10.66 10.67
    lon_bnds        (lon, bnds) float64 dask.array<chunksize=(10240, 2), meta=np.ndarray>
  * time            (time) datetime64[ns] 2017-01-01T09:57:21.976999680 ... 2...
    time_bnds       (time, bnds) datetime64[ns] dask.array<chunksize=(1479, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    chl_c2rcc       (time, lat, lon) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    tsm_c2rcc       (time, lat, lon) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
    tur_nechad_665  (time, lat, lon) float64 dask.array<chunksize=(1, 1024, 1024), meta=np.ndarray>
Attributes:
    acknowledgment:             ESA Sentinel-3 OLCI data, EU HIGHROC project
    comment:                    
    contributor_name:           
    contributor_role:           
    creator_email:              info@brockmann-consult.de
    creator_name:               Brockmann Consult GmbH
    creator_url:                https://www.brockmann-consult.de
    date_modified:              2019-09-12T08:09:32.908854
    geospatial_lat_max:         62.666666666666664
    geospatial_lat_min:         48.0
    geospatial_lat_resolution:  0.002604166666666666
    geospatial_lat_units:       degrees_north
    geospatial_lon_max:         10.666666666666664
    geospatial_lon_min:         -16.0
    geospatial_lon_resolution:  0.0026041666666666665
    geospatial_lon_units:       degrees_east
    history:                    xcube/reproj-snap-nc
    id:                         dcs4cop-bc-olci-sns-l2c-v1
    institution:                Brockmann Consult GmbH
    keywords:                   
    license:                    terms and conditions of the DCS4COP data dist...
    naming_authority:           bc
    processing_level:           L2C
    project:                    DCS4COP
    publisher_email:            info@brockmann-consult.de
    publisher_name:             Brockmann Consult GmbH
    publisher_url:              https://www.brockmann-consult.de
    references:                 https://dcs4cop.eu/
    source:                     Sentinel-3 OLCI L2 surface observation
    standard_name_vocabulary:   
    summary:                    
    title:                      DCS4COP Sentinel-3 OLCI L2C Data Cube

Extract a time-series at a point of interes, drop missing values:

In [3]:
chl_ts = cube.chl_c2rcc.sel(lat=53.890319, lon=6.693590, method='nearest').dropna(dim='time').load()
chl_ts

FileNotFoundError: The specified key does not exist.

In [ ]:
chl_ts.plot.line(marker='x', figsize=(20, 5), ylim=(0, 12))

In [ ]:
chl_ts.time

Plot the image at one of the times filtered out above:

In [ ]:
cube.chl_c2rcc.sel(time=np.datetime64('2017-04-11 10:00'), method='nearest').plot.imshow(vmax=18, figsize=(16, 16))


---

## Create monthly CHL aggregations for specific regions

Read Shapefile with Northsea regions according to Water Framework Directive (WFD):

In [ ]:
regions = gpd.read_file('WRRL_Klassen_Nordsee/WRRL_Klassen_Nordsee_latlon3.shp')
regions

In [ ]:
IPython.display.GeoJSON(regions.__geo_interface__)

The remaining steps can be done in a loop over all regions. However, for this demonstration we just pick a single random region:

In [ ]:
# region = regions.loc[regions['NAME'] == 'Küstenmeer Elbe']
# region = regions.loc[regions['NAME'] == 'Küstenmeer Eider']
region = regions.loc[regions['NAME'] == 'Küstenmeer Ems']
region

In [ ]:
IPython.display.GeoJSON(region.__geo_interface__)

Note, this reagion is still a data frame, whose geometry is a one-element series. That's is how we get the actual geometry object: 

In [ ]:
polygon = region.geometry.values[0]
polygon

We'll now narrow down the data cube to that selected region and mask out all values that don't intersect with the region polygon:

In [ ]:
cube_masked = mask_dataset_by_geometry(cube, polygon)
cube_masked

Next, we resample the masked and clipped cube to monthly averages:

In [ ]:
cube_1m = resample_in_time(cube_masked, frequency='1M', method='mean')
cube_1m

In [ ]:
cube_1m.time

We will frequently use the data `cube_1m` in the following steps, so that we benefit if we load them all into memory:

In [ ]:
cube_1m = cube_1m.load()

We are going to work with the CHL values only, therefore:

In [ ]:
chl = cube_1m.chl_c2rcc_mean

In [ ]:
chl.plot.imshow(col='time', col_wrap=4, vmax=15)

Now we can compute time series of the mean, median and the p90 values:

In [ ]:
chl_mean = chl.mean(dim=['lat', 'lon'])
chl_p50 = chl.median(dim=['lat', 'lon'])
chl_p90 = chl.quantile(0.9, dim=['lat', 'lon'])

In [ ]:
chl_ts_ds = xr.Dataset(dict(chl_mean=chl_mean, chl_p50=chl_p50, chl_p90=chl_p90))
chl_ts_df = chl_ts_ds.to_dataframe().drop('quantile', axis=1)
chl_ts_df

In [ ]:
chl_ts_df.plot.line(figsize=(16,10), marker='x')

---

## Compute monthly CHL "anomalies" 

The following steps explain, how to compute the "anomaly" of every month for the selected region with respect to the mean of all months in 2017:

In [ ]:
chl_mean = chl.mean(dim='time')
chl_mean

In [ ]:
chl_mean.plot.imshow(vmax=15, figsize=(16,10))

In [ ]:
chl_anomaly = chl - chl_mean

In [ ]:
chl_anomaly.plot.imshow(col='time', col_wrap=4, cmap='bwr', vmin=-5, vmax=5)

---

## Compare Cube CHL with In-Situ CHL

The following analysis provided here represents a validation of the cube data with in-situ data.

Read some in-situ data points from a shiptrack made by BSH in Summer 2017: 

In [ ]:
cruise = pd.read_csv('./summercruise_2017_positions_for_pixex.txt', delimiter='\t', converters={'DateTime': np.datetime64})
cruise

Since we want to compare the points with data in the cube, we need to match the names of coordinate variables first, so we rename the columns:

In [ ]:
cruise = cruise.rename(columns={'longitude': 'lon', 'latitude': 'lat', 'DateTime': 'time'})
cruise['ID'] = np.arange(0, len(cruise))

Where are these points?

In [ ]:
from shapely.geometry import Point
cruise_geom = gpd.GeoDataFrame(cruise.drop(['lon', 'lat'], axis=1), crs={'init': 'epsg:4326'}, geometry=[Point(xy) for xy in zip(cruise.lon, cruise.lat)])
IPython.display.GeoJSON(cruise_geom.__geo_interface__)

We now select that month from the cube (no need to do this, this is an optional step for demonstration only):

In [ ]:
cube_subset = cube.sel(time='2017-08')
cube_subset

We now resample that month to weekly averages in order to increase the probability that we find match-ups:

In [ ]:
cube_1w = resample_in_time(cube_subset, frequency='1W', method='mean')
cube_1w

Next, we extract values from the data cube `cube_1w` for all the points in the `cruise`:

In [ ]:
match_ups = get_cube_values_for_points(cube_1w, cruise, include_coords=True, include_indexes=True)
match_ups

Here are the match-ups for CHL and TUR:

In [ ]:
chl_match_ups = match_ups.chl_c2rcc_mean.values
tur_match_ups = match_ups.tur_nechad_665_mean.values

We add them to the cruise data:

In [ ]:
cruise['chl_olci'] = chl_match_ups
cruise['tur_olci'] = tur_match_ups / 40
cruise

In [ ]:
cruise.plot.scatter('Chlorophyll_ges', 'chl_olci', xlim=(0, 10), ylim=(0, 10), figsize=(5,5))

In [ ]:
cruise.plot.scatter('TUR', 'tur_olci', xlim=(0, 0.5), ylim=(0, 0.5), figsize=(5,5))